In [5]:
import os
import sys
import gensim
import pandas as pd
from gensim.models.doc2vec import LabeledSentence
csv = 'corpus_formalized_cleaned.csv'
data = pd.read_csv(csv)
data.head()

,content,polarity
0,<number> 000 bisa maksimal yang dapat anda me...,negative
1,<number> 1 per <number> jam menunggu untuk di...,negative
2,<number> 15 tahun yang lalu nero adalah di ba...,negative
3,<number> 2 <number> april <number> room <numb...,negative
4,<number> 2 <number> april <number> room <numb...,negative


In [9]:
def sentiment_label(polarity):
    if polarity=='negative':
        return 0
    else:
        return 1

In [12]:
data['sentiment'] = data['polarity'].apply(sentiment_label)
data.head()

,content,polarity,sentiment
0,<number> 000 bisa maksimal yang dapat anda me...,negative,0
1,<number> 1 per <number> jam menunggu untuk di...,negative,0
2,<number> 15 tahun yang lalu nero adalah di ba...,negative,0
3,<number> 2 <number> april <number> room <numb...,negative,0
4,<number> 2 <number> april <number> room <numb...,negative,0


In [2]:
from sklearn.cross_validation import train_test_split
SEED = 2000

x_train, x_validation, y_train, y_validation = train_test_split(data.SentimentText, data.Sentiment, test_size=.2, random_state=SEED)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
all_x = pd.concat([x_train,x_validation])

all_x_w2v = labelize_text(all_x, 'ALL')
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [5]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np

model_w2v = Word2Vec(size=200, min_count=10)
model_w2v.build_vocab([x.words for x in tqdm(all_x_w2v)])
model_w2v.train([x.words for x in tqdm(all_x_w2v)], total_examples=len(all_x_w2v), epochs=1)

100%|████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 991411.23it/s]


(3005646, 3344822)

In [6]:
model_w2v.most_similar('good')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('pretty', 0.9661742448806763),
 ('bad', 0.9536818265914917),
 ('funny', 0.9490100741386414),
 ('quite', 0.9480053186416626),
 ('overall', 0.946625828742981),
 ('decent', 0.9436829090118408),
 ('awful', 0.9410758018493652),
 ('well', 0.9297658801078796),
 ('too', 0.9243580102920532),
 ('entertaining', 0.9219523668289185)]

In [7]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

word_vec = [model_w2v[w] for w in list(model_w2v.wv.vocab.keys())[:5000]]

from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vec)

tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(model_w2v.wv.vocab.keys())[:5000]

plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.130s...
[t-SNE] Computed neighbors for 5000 samples in 8.753s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.052404
[t-SNE] KL divergence after 250 iterations with early exaggeration: 70.726913
[t-SNE] Error after 1000 iterations: 1.261537


## Average Vector With TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in all_x_w2v])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

def build_Word_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: 
            
            continue
    if count != 0:
        vec /= count
    return vec

In [9]:
from sklearn.preprocessing import scale
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from time import time
from sklearn.metrics import roc_curve

acc=[]
times=[]
fp = []
tp = []

train_vecs_w2v = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)
val_vecs_w2v = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_validation))])
val_vecs_w2v = scale(val_vecs_w2v)


model = Sequential()
model.add(Dense(256, activation='relu', input_dim=200))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
t0 = time()
model.fit(train_vecs_w2v, y_train, epochs=100, batch_size=32, verbose=2)
score = model.evaluate(val_vecs_w2v, y_validation, batch_size=128, verbose=2)
train_val_time = time()-t0

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
20000it [00:57, 346.38it/s]
5000it [00:14, 355.69it/s]


Epoch 1/100
 - 5s - loss: 0.5783 - acc: 0.7014
Epoch 2/100
 - 3s - loss: 0.5409 - acc: 0.7298
Epoch 3/100
 - 3s - loss: 0.5231 - acc: 0.7459
Epoch 4/100
 - 3s - loss: 0.5163 - acc: 0.7489
Epoch 5/100
 - 3s - loss: 0.5091 - acc: 0.7561
Epoch 6/100
 - 3s - loss: 0.5026 - acc: 0.7563
Epoch 7/100
 - 3s - loss: 0.5022 - acc: 0.7569
Epoch 8/100
 - 3s - loss: 0.4962 - acc: 0.7643
Epoch 9/100
 - 3s - loss: 0.4957 - acc: 0.7636
Epoch 10/100
 - 3s - loss: 0.4909 - acc: 0.7665
Epoch 11/100
 - 3s - loss: 0.4890 - acc: 0.7682
Epoch 12/100
 - 3s - loss: 0.4862 - acc: 0.7686
Epoch 13/100
 - 3s - loss: 0.4840 - acc: 0.7702
Epoch 14/100
 - 3s - loss: 0.4815 - acc: 0.7734
Epoch 15/100
 - 3s - loss: 0.4784 - acc: 0.7755
Epoch 16/100
 - 3s - loss: 0.4760 - acc: 0.7756
Epoch 17/100
 - 3s - loss: 0.4737 - acc: 0.7769
Epoch 18/100
 - 3s - loss: 0.4732 - acc: 0.7781
Epoch 19/100
 - 3s - loss: 0.4688 - acc: 0.7799
Epoch 20/100
 - 3s - loss: 0.4674 - acc: 0.7803
Epoch 21/100
 - 3s - loss: 0.4644 - acc: 0.7820
E

In [10]:
y_pred= model.predict(val_vecs_w2v).ravel()
fpr, tpr, _ = roc_curve(y_validation, y_pred)

print (score[1])
fp = []
tp = []
fp.append(fpr)
tp.append(tpr)
acc.append(score[1]*100)
times.append(train_val_time)

0.7254


## Distributed Bag Of Words - DBOW

In [11]:
from gensim.models import Doc2Vec
import multiprocessing

cores = multiprocessing.cpu_count()
model_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_x_w2v)])
model_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_dbow[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

train_vecs_dbow = np.concatenate([build_doc_Vector(z, 100) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_dbow = scale(train_vecs_dbow)
val_vecs_dbow = np.concatenate([build_doc_Vector(z, 100) for z in tqdm(map(lambda x: x.words, x_validation))])
val_vecs_dbow = scale(val_vecs_dbow)

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=100))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

t0 = time()
model.fit(train_vecs_dbow, y_train, epochs=100, batch_size=32, verbose=2)
score = model.evaluate(val_vecs_dbow, y_validation, batch_size=128, verbose=2)
train_val_time = time()-t0

C:\Users\User\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|███████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 1303599.09it/s]
20000it [00:38, 513.14it/s]
5000it [00:09, 514.70it/s]


Epoch 1/100
 - 3s - loss: 0.6417 - acc: 0.6406
Epoch 2/100
 - 3s - loss: 0.5905 - acc: 0.6905
Epoch 3/100
 - 3s - loss: 0.5501 - acc: 0.7222
Epoch 4/100
 - 3s - loss: 0.4902 - acc: 0.7668
Epoch 5/100
 - 3s - loss: 0.4191 - acc: 0.8100
Epoch 6/100
 - 3s - loss: 0.3398 - acc: 0.8554
Epoch 7/100
 - 3s - loss: 0.2601 - acc: 0.8945
Epoch 8/100
 - 3s - loss: 0.1867 - acc: 0.9321
Epoch 9/100
 - 3s - loss: 0.1339 - acc: 0.9557
Epoch 10/100
 - 3s - loss: 0.0899 - acc: 0.9769
Epoch 11/100
 - 3s - loss: 0.0661 - acc: 0.9834
Epoch 12/100
 - 3s - loss: 0.0648 - acc: 0.9810
Epoch 13/100
 - 3s - loss: 0.0788 - acc: 0.9728
Epoch 14/100
 - 3s - loss: 0.0426 - acc: 0.9893
Epoch 15/100
 - 3s - loss: 0.0283 - acc: 0.9946
Epoch 16/100
 - 3s - loss: 0.0512 - acc: 0.9834
Epoch 17/100
 - 3s - loss: 0.0562 - acc: 0.9811
Epoch 18/100
 - 3s - loss: 0.0389 - acc: 0.9873
Epoch 19/100
 - 3s - loss: 0.0395 - acc: 0.9864
Epoch 20/100
 - 3s - loss: 0.0364 - acc: 0.9883
Epoch 21/100
 - 3s - loss: 0.0369 - acc: 0.9885
E

In [12]:
y_pred= model.predict(val_vecs_dbow).ravel()
fpr, tpr, _ = roc_curve(y_validation, y_pred)

print (score[1])
fp.append(fpr)
tp.append(tpr)
acc.append(score[1]*100)
times.append(train_val_time)

0.637


## Distributed Memory Concatenation - DMC

In [13]:
cores = multiprocessing.cpu_count()
model_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_dmc.build_vocab([x for x in tqdm(all_x_w2v)])
model_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_dmc[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

  
train_vecs_dmc = np.concatenate([build_doc_Vector(z, 100) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_dmc = scale(train_vecs_dmc)


val_vecs_dmc = np.concatenate([build_doc_Vector(z, 100) for z in tqdm(map(lambda x: x.words, x_validation))])
val_vecs_dmc = scale(val_vecs_dmc)

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=100))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

t0 = time()
model.fit(train_vecs_dmc, y_train, epochs=100, batch_size=32, verbose=2)
score = model.evaluate(val_vecs_dmc, y_validation, batch_size=128, verbose=2)
train_val_time = time()-t0

y_pred= model.predict(val_vecs_dmc).ravel()
fpr, tpr, _ = roc_curve(y_validation, y_pred)

print (score[1])
fp.append(fpr)
tp.append(tpr)
acc.append(score[1]*100)
times.append(train_val_time)

C:\Users\User\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 956782.31it/s]
20000it [00:38, 517.84it/s]
5000it [00:09, 516.69it/s]


Epoch 1/100
 - 3s - loss: 0.4290 - acc: 0.8046
Epoch 2/100
 - 3s - loss: 0.3877 - acc: 0.8283
Epoch 3/100
 - 3s - loss: 0.3651 - acc: 0.8402
Epoch 4/100
 - 3s - loss: 0.3403 - acc: 0.8515
Epoch 5/100
 - 3s - loss: 0.3072 - acc: 0.8687
Epoch 6/100
 - 3s - loss: 0.2700 - acc: 0.8861
Epoch 7/100
 - 3s - loss: 0.2297 - acc: 0.9070
Epoch 8/100
 - 3s - loss: 0.1881 - acc: 0.9258
Epoch 9/100
 - 3s - loss: 0.1464 - acc: 0.9440
Epoch 10/100
 - 3s - loss: 0.1137 - acc: 0.9585
Epoch 11/100
 - 3s - loss: 0.0837 - acc: 0.9709
Epoch 12/100
 - 3s - loss: 0.0681 - acc: 0.9782
Epoch 13/100
 - 3s - loss: 0.0515 - acc: 0.9842
Epoch 14/100
 - 3s - loss: 0.0431 - acc: 0.9865
Epoch 15/100
 - 3s - loss: 0.0489 - acc: 0.9825
Epoch 16/100
 - 3s - loss: 0.0436 - acc: 0.9864
Epoch 17/100
 - 3s - loss: 0.0358 - acc: 0.9886
Epoch 18/100
 - 3s - loss: 0.0305 - acc: 0.9900
Epoch 19/100
 - 3s - loss: 0.0185 - acc: 0.9952
Epoch 20/100
 - 3s - loss: 0.0239 - acc: 0.9932
Epoch 21/100
 - 3s - loss: 0.0531 - acc: 0.9816
E

## Distributed Memory Mean - DMM

In [14]:
cores = multiprocessing.cpu_count()
model_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(all_x_w2v)])
model_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)

def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_dmm[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

  
train_vecs_dmm = np.concatenate([build_doc_Vector(z, 100) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_dmm = scale(train_vecs_dmm)

val_vecs_dmm = np.concatenate([build_doc_Vector(z, 100) for z in tqdm(map(lambda x: x.words, x_validation))])
val_vecs_dmm = scale(val_vecs_dmm)

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=100))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

t0 = time()
model.fit(train_vecs_dmm, y_train, epochs=100, batch_size=32, verbose=2)
score = model.evaluate(val_vecs_dmm, y_validation, batch_size=128, verbose=2)
train_val_time = time()-t0

y_pred= model.predict(val_vecs_dmm).ravel()
fpr, tpr, _ = roc_curve(y_validation, y_pred)

print (score[1])
fp.append(fpr)
tp.append(tpr)
acc.append(score[1]*100)
times.append(train_val_time)

C:\Users\User\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 872729.69it/s]
20000it [00:38, 519.56it/s]
5000it [00:09, 519.73it/s]


Epoch 1/100
 - 3s - loss: 0.3980 - acc: 0.8237
Epoch 2/100
 - 3s - loss: 0.3625 - acc: 0.8421
Epoch 3/100
 - 3s - loss: 0.3501 - acc: 0.8492
Epoch 4/100
 - 3s - loss: 0.3337 - acc: 0.8537
Epoch 5/100
 - 3s - loss: 0.3182 - acc: 0.8635
Epoch 6/100
 - 3s - loss: 0.2965 - acc: 0.8729
Epoch 7/100
 - 3s - loss: 0.2740 - acc: 0.8817
Epoch 8/100
 - 3s - loss: 0.2475 - acc: 0.8979
Epoch 9/100
 - 3s - loss: 0.2177 - acc: 0.9113
Epoch 10/100
 - 3s - loss: 0.1903 - acc: 0.9231
Epoch 11/100
 - 3s - loss: 0.1653 - acc: 0.9343
Epoch 12/100
 - 3s - loss: 0.1371 - acc: 0.9456
Epoch 13/100
 - 3s - loss: 0.1136 - acc: 0.9580
Epoch 14/100
 - 3s - loss: 0.0922 - acc: 0.9660
Epoch 15/100
 - 3s - loss: 0.0780 - acc: 0.9725
Epoch 16/100
 - 3s - loss: 0.0647 - acc: 0.9778
Epoch 17/100
 - 3s - loss: 0.0525 - acc: 0.9832
Epoch 18/100
 - 3s - loss: 0.0522 - acc: 0.9834
Epoch 19/100
 - 3s - loss: 0.0396 - acc: 0.9873
Epoch 20/100
 - 3s - loss: 0.0506 - acc: 0.9834
Epoch 21/100
 - 3s - loss: 0.0396 - acc: 0.9875
E

## Combined Model DBOW + DMC

In [15]:
def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmc[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

train_vecs_dbow_dmc = np.concatenate([build_doc_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_dbow_dmc = scale(train_vecs_dbow_dmc)

val_vecs_dbow_dmc = np.concatenate([build_doc_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_validation))])
val_vecs_dbow_dmc = scale(val_vecs_dbow_dmc)
                             
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=200))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

t0 = time()
model.fit(train_vecs_dbow_dmc, y_train, epochs=100, batch_size=32, verbose=2)
score = model.evaluate(val_vecs_dbow_dmc, y_validation, batch_size=128, verbose=2)
train_val_time = time()-t0

y_pred= model.predict(val_vecs_dbow_dmc).ravel()
fpr, tpr, _ = roc_curve(y_validation, y_pred)

print (score[1])
fp.append(fpr)
tp.append(tpr)
acc.append(score[1]*100)
times.append(train_val_time)

20000it [01:26, 230.76it/s]
5000it [00:21, 232.58it/s]


Epoch 1/100
 - 4s - loss: 0.4337 - acc: 0.8042
Epoch 2/100
 - 3s - loss: 0.3588 - acc: 0.8414
Epoch 3/100
 - 3s - loss: 0.3123 - acc: 0.8672
Epoch 4/100
 - 3s - loss: 0.2495 - acc: 0.8964
Epoch 5/100
 - 3s - loss: 0.1822 - acc: 0.9271
Epoch 6/100
 - 3s - loss: 0.1235 - acc: 0.9553
Epoch 7/100
 - 3s - loss: 0.0862 - acc: 0.9692
Epoch 8/100
 - 3s - loss: 0.0555 - acc: 0.9819
Epoch 9/100
 - 3s - loss: 0.0455 - acc: 0.9851
Epoch 10/100
 - 3s - loss: 0.0438 - acc: 0.9850
Epoch 11/100
 - 3s - loss: 0.0447 - acc: 0.9858
Epoch 12/100
 - 3s - loss: 0.0319 - acc: 0.9887
Epoch 13/100
 - 3s - loss: 0.0424 - acc: 0.9852
Epoch 14/100
 - 3s - loss: 0.0217 - acc: 0.9927
Epoch 15/100
 - 3s - loss: 0.0362 - acc: 0.9867
Epoch 16/100
 - 3s - loss: 0.0295 - acc: 0.9898
Epoch 17/100
 - 3s - loss: 0.0176 - acc: 0.9948
Epoch 18/100
 - 3s - loss: 0.0238 - acc: 0.9917
Epoch 19/100
 - 3s - loss: 0.0315 - acc: 0.9883
Epoch 20/100
 - 3s - loss: 0.0269 - acc: 0.9903
Epoch 21/100
 - 3s - loss: 0.0139 - acc: 0.9954
E

## Combined Model DBOW + DMM

In [16]:
def build_doc_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += np.append(model_dbow[word] * tfidf[word], model_dmm[word] * tfidf[word])
            count += 1
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    return vec

train_vecs_dbow_dmm = np.concatenate([build_doc_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_train))])
train_vecs_dbow_dmm = scale(train_vecs_dbow_dmm)

val_vecs_dbow_dmm = np.concatenate([build_doc_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_validation))])
val_vecs_dbow_dmm = scale(val_vecs_dbow_dmm)

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=200))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

t0 = time()
model.fit(train_vecs_dbow_dmm, y_train, epochs=100, batch_size=32, verbose=2)
score = model.evaluate(val_vecs_dbow_dmm, y_validation, batch_size=128, verbose=2)
train_val_time = time()-t0

y_pred= model.predict(val_vecs_dbow_dmm).ravel()
fpr, tpr, _ = roc_curve(y_validation, y_pred)

print (score[1])
fp.append(fpr)
tp.append(tpr)
acc.append(score[1]*100)
times.append(train_val_time)

20000it [01:26, 230.41it/s]
5000it [00:21, 232.36it/s]


Epoch 1/100
 - 4s - loss: 0.4047 - acc: 0.8206
Epoch 2/100
 - 3s - loss: 0.3436 - acc: 0.8527
Epoch 3/100
 - 3s - loss: 0.3031 - acc: 0.8704
Epoch 4/100
 - 3s - loss: 0.2567 - acc: 0.8914
Epoch 5/100
 - 3s - loss: 0.2040 - acc: 0.9175
Epoch 6/100
 - 3s - loss: 0.1516 - acc: 0.9422
Epoch 7/100
 - 3s - loss: 0.1024 - acc: 0.9635
Epoch 8/100
 - 3s - loss: 0.0720 - acc: 0.9752
Epoch 9/100
 - 3s - loss: 0.0545 - acc: 0.9815
Epoch 10/100
 - 3s - loss: 0.0461 - acc: 0.9851
Epoch 11/100
 - 3s - loss: 0.0446 - acc: 0.9846
Epoch 12/100
 - 3s - loss: 0.0334 - acc: 0.9892
Epoch 13/100
 - 3s - loss: 0.0279 - acc: 0.9910
Epoch 14/100
 - 3s - loss: 0.0347 - acc: 0.9879
Epoch 15/100
 - 3s - loss: 0.0355 - acc: 0.9879
Epoch 16/100
 - 3s - loss: 0.0208 - acc: 0.9934
Epoch 17/100
 - 3s - loss: 0.0256 - acc: 0.9902
Epoch 18/100
 - 3s - loss: 0.0322 - acc: 0.9876
Epoch 19/100
 - 3s - loss: 0.0196 - acc: 0.9934
Epoch 20/100
 - 3s - loss: 0.0241 - acc: 0.9920
Epoch 21/100
 - 3s - loss: 0.0319 - acc: 0.9891
E

In [17]:
acc

[72.54, 63.7, 79.17999999999999, 81.62, 79.94, 81.44]

In [19]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import numpy as np


names = ["WORD2VEC","DBOW", "DMC", "DMM", "DBOW+DMC", "DBOW+DMM"]

trace1 = go.Bar(
    x=names,
    y=acc,
    name='Accuracy percentage'
)
trace2 = go.Bar(
    x=names,
    y=times,
    name='train and test time'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

In [20]:
data = []
for i in range(0, len(fp)):
    trace = go.Scatter(x=fp[i], y=tp[i],
                        mode='lines', 
                        name='ROC curve {}'
                               ''.format(names[i]))
    data.append(trace)
    
layout = go.Layout(title='Receiver operating characteristic ',
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'))

fig = go.Figure(data=data, layout=layout)
    
py.iplot(fig)